In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from xgboost import XGBRegressor
from scipy.stats import randint

from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split


In [2]:
# Importer le DataFrame propre depuis le fichier CSV
df = pd.read_csv('data.csv')
df.head(4)
df.columns

Index(['LoanNr_ChkDgt', 'Name', 'City', 'State', 'Zip', 'Bank', 'BankState',
       'NAICS', 'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist',
       'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr',
       'LowDoc', 'ChgOffDate', 'DisbursementDate', 'DisbursementGross',
       'BalanceGross', 'MIS_Status', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv'],
      dtype='object')

In [5]:
# Remplacer les valeurs non numériques par 0 et le type des variables
df['NewExist'] = df['NewExist'].fillna(0)
df['UrbanRural'] = df['UrbanRural'].fillna(0)


# Convertir la colonne en type entier
df['NewExist'] = df['NewExist'].astype(int)
df['NewExist'].astype(int)

df['UrbanRural'] = df['UrbanRural'].astype(int)
df['UrbanRural'].astype(int)

#conserver les deux premiers chiffres de naics
df['NAICS_digit'] = df['NAICS'].astype(str).str[:2]

df.dtypes

LoanNr_ChkDgt          int64
Name                  object
City                  object
State                 object
Zip                    int64
Bank                  object
BankState             object
NAICS                  int64
ApprovalDate          object
ApprovalFY           float64
Term                   int64
NoEmp                  int64
NewExist               int64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int64
UrbanRural             int64
RevLineCr             object
LowDoc                object
ChgOffDate            object
DisbursementDate      object
DisbursementGross    float64
BalanceGross         float64
MIS_Status             int64
ChgOffPrinGr         float64
GrAppv               float64
SBA_Appv             float64
NAICS_digit           object
dtype: object

In [4]:
#Supprimer les colonnes inutiles
#df = df.drop(['ApprovalFY', 'ApprovalDate'], axis=1)

#traiter la colonen NAICS pour qu'elle ne contienne que les 2 premiers chiffres des valeurs NAICS
#df['NAICS_digit'] = (df['NAICS'] / 10000 ).astype(int)
#df = df.drop(['NAICS'], axis=1)
df.columns
# Utilisez l'attribut shape pour obtenir le nombre de lignes et de colonnes de votre dataframe
nombre_lignes, nombre_colonnes = df.shape

print("Nombre de lignes :", nombre_lignes)
print("Nombre de colonnes :", nombre_colonnes)


Nombre de lignes : 897167
Nombre de colonnes : 28


### Modélisation

on teste une modélisation avec une imputation des valeurs manquantes, et une en supprimant les valeurs manquantes du df. Selon les résultats, on optera pour l'une ou l'autre du choix.

#### 1. RandomForest Simple avec SimpleImputer

In [5]:
# Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz
df.columns

Index(['LoanNr_ChkDgt', 'Name', 'City', 'State', 'Zip', 'Bank', 'BankState',
       'NAICS', 'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist',
       'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr',
       'LowDoc', 'ChgOffDate', 'DisbursementDate', 'DisbursementGross',
       'BalanceGross', 'MIS_Status', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv',
       'NAICS_digit'],
      dtype='object')

In [6]:
#separer dataset en features et target
X = df.drop('MIS_Status', axis=1)
y = df['MIS_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, test_size=0.05, random_state=42, stratify= y)

cat_col = ['RevLineCr', 'LowDoc', 'FranchiseCode', 'UrbanRural', 'NewExist']
num_col = ['NAICS_digit', 'Term' ]

In [22]:
# Test RandomForest sans hyperparamètres 
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop= 'if_binary'))
])

num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_col),
        ('num', num_transformer, num_col)
    ])

# Création du pipeline : prétraitement + modèle RandomForest
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier())])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

# performance du modèle
score_tr = pipeline.score(X_train, y_train)
score_te = pipeline.score(X_test, y_test)

# importance des caractéristiques
feature_importance = pipeline.named_steps['classifier'].feature_importances_
print("Importance des caractéristiques :", feature_importance)

print('_____________________')
print("Score du modèle (train) :", score_tr)
print("Score du modèle (test) :", score_te)
print('_____________________')

#matrice confusion
conf_matrix = confusion_matrix(y_test, y_pred)
print('_____________________')

print("Métrique pour le modèle RandomForest Simple")
print("Score d'accuracy", accuracy_score(y_test, y_pred))
print("Score du recall : ", recall_score(y_test, y_pred))
print("Score de la precision : ", precision_score(y_test, y_pred))
print("Score F1 : ", f1_score(y_test, y_pred))
print("")
print(classification_report(y_test, y_pred))

ValueError: A given column is not a column of the dataframe

In [7]:
#on supprime les valeurs manquantes nan de notre df puis on applique une modélisation

missing_rows = df.isnull().any(axis=1).sum()
missing_rows

df_sans_valeurs_manquantes = df.dropna()


In [18]:
#refait modelisation sur df_sans_valeurs_manquantes sans valeurs manquantes
#separer dataset en features et target
X = df_sans_valeurs_manquantes.drop('MIS_Status', axis=1)
y = df_sans_valeurs_manquantes['MIS_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, test_size=0.05, random_state=42, stratify= y)

cat_col = ['RevLineCr', 'LowDoc', 'FranchiseCode', 'UrbanRural', 'NewExist']
num_col = ['NAICS_digit', 'Term' ]

In [19]:
# Test RandomForest sans hyperparamètres 
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop= 'if_binary'))
])

num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_col),
        ('num', num_transformer, num_col)
    ])

# Création du pipeline : prétraitement + modèle RandomForest
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier())])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

# performance du modèle
score_tr = pipeline.score(X_train, y_train)
score_te = pipeline.score(X_test, y_test)

# importance des caractéristiques
feature_importance = pipeline.named_steps['classifier'].feature_importances_
print("Importance des caractéristiques :", feature_importance)

print('_____________________')
print("Score du modèle (train) :", score_tr)
print("Score du modèle (test) :", score_te)
print('_____________________')

#matrice confusion
conf_matrix = confusion_matrix(y_test, y_pred)
print('_____________________')

print("Métrique pour le modèle RandomForest Simple sur df sans valeurs manquantes")
print("Score d'accuracy", accuracy_score(y_test, y_pred))
print("Score du recall : ", recall_score(y_test, y_pred))
print("Score de la precision : ", precision_score(y_test, y_pred))
print("Score F1 : ", f1_score(y_test, y_pred))
print("")
print(classification_report(y_test, y_pred))

Importance des caractéristiques : [1.21888568e-02 5.62935609e-03 7.35786542e-03 3.13994680e-03
 3.13834968e-03 3.36263484e-03 4.61852904e-04 2.70303161e-03
 2.88109041e-03 2.55470815e-01 7.03666201e-01]
_____________________
Score du modèle (train) : 0.9738984054164046
Score du modèle (test) : 0.9690739764540502
_____________________
_____________________
Métrique pour le modèle RandomForest Simple sur df sans valeurs manquantes
Score d'accuracy 0.9690739764540502
Score du recall :  0.9972870320130223
Score de la precision :  0.9716299559471365
Score F1 :  0.9842913245269547

              precision    recall  f1-score   support

           0       0.06      0.01      0.01       162
           1       0.97      1.00      0.98      5529

    accuracy                           0.97      5691
   macro avg       0.52      0.50      0.50      5691
weighted avg       0.95      0.97      0.96      5691



#### 2.Testons KNNImputer

In [27]:
# Test KNNImputer
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop = 'if_binary'))
])

num_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors = 3, weights = 'uniform')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_col),
        ('num', num_transformer, num_col)
    ])

# Création du pipeline : prétraitement + modèle RandomForest
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier())])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

# performance du modèle
score_tr = pipeline.score(X_train, y_train)
score_te = pipeline.score(X_test, y_test)

print("Score du modèle (train) :", score_tr)
print("Score du modèle (test) :", score_te)

# matrice confusion
conf_matrix = confusion_matrix(y_test, y_pred)

print("Métrique pour le modèle AdaboostClassifierr")
print("Score d'accuracy", accuracy_score(y_test, y_pred))
print("Score du recall : ", recall_score(y_test, y_pred))
print("Score de la precision : ", precision_score(y_test, y_pred))
print("Score F1 : ", f1_score(y_test, y_pred))
print("")
print(classification_report(y_test, y_pred))

Score du modèle (train) : 0.9467575101958446
Score du modèle (test) : 0.927104928776834
Métrique pour le modèle AdaboostClassifierr
Score d'accuracy 0.927104928776834
Score du recall :  0.764026402640264
Score de la precision :  0.8100942126514132
Score F1 :  0.7863862032923961

              precision    recall  f1-score   support

           0       0.95      0.96      0.96     36981
           1       0.81      0.76      0.79      7878

    accuracy                           0.93     44859
   macro avg       0.88      0.86      0.87     44859
weighted avg       0.93      0.93      0.93     44859



In [16]:
# Tests KNNImputer pour les variables numeriques
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop = 'if_binary'))
])

num_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors = 3, weights = 'uniform')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_col),
        ('num', num_transformer, num_col)
    ])

# Création du pipeline : prétraitement + modèle RandomForest
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier(n_estimators=80, max_depth=40, min_samples_leaf=5, max_features='sqrt'))])

pipeline.fit(X_train, y_train)

# performance du modèle
score_tr = pipeline.score(X_train, y_train)
score_te = pipeline.score(X_test, y_test)

print("Score du modèle (train) :", score_tr)
print("Score du modèle (test) :", score_te)


Score du modèle (train) : 0.8975565050467521
Score du modèle (test) : 0.8926178985030707


In [33]:
#on joue avec les hyperparametre

# Tests KNNImputer pour les variables numeriques
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop= 'if_binary'))
])

num_transformer = Pipeline(steps=[
    ('imputer', IterativeImputer (max_iter=10, random_state=0)),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_col),
        ('num', num_transformer, num_col)
    ])

# Création du pipeline : prétraitement + modèle RandomForest
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier(n_estimators= 80, max_depth=40, min_samples_leaf=5, max_features='sqrt'))])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)


# performance du modèle
score_tr = pipeline.score(X_train, y_train)
score_te = pipeline.score(X_test, y_test)

print("Score du modèle (train) :", score_tr)
print("Score du modèle (test) :", score_te)

# matrice confusion
conf_matrix = confusion_matrix(y_test, y_pred)

print("Métrique pour le modèle AdaboostClassifierr")
print("Score d'accuracy", accuracy_score(y_test, y_pred))
print("Score du recall : ", recall_score(y_test, y_pred))
print("Score de la precision : ", precision_score(y_test, y_pred))
print("Score F1 : ", f1_score(y_test, y_pred))
print("")
print(classification_report(y_test, y_pred))

/home/utilisateur/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/home/utilisateur/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Score du modèle (train) : 0.8981764808027145
Score du modèle (test) : 0.8956508170043915
Métrique pour le modèle AdaboostClassifierr
Score d'accuracy 0.8956508170043915
Score du recall :  0.46585427773546584
Score de la precision :  0.8858315230509293
Score F1 :  0.6105981199567423

              precision    recall  f1-score   support

           0       0.90      0.99      0.94     36981
           1       0.89      0.47      0.61      7878

    accuracy                           0.90     44859
   macro avg       0.89      0.73      0.78     44859
weighted avg       0.89      0.90      0.88     44859



### Modélisation 

#### 1. Randomized search + RandomForest

In [16]:
#separer dataset en features et target
X = df.drop('MIS_Status', axis=1)
y = df['MIS_Status']
df.columns

Index(['LoanNr_ChkDgt', 'Name', 'City', 'State', 'Zip', 'Bank', 'BankState',
       'NAICS', 'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist',
       'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr',
       'LowDoc', 'ChgOffDate', 'DisbursementDate', 'DisbursementGross',
       'BalanceGross', 'MIS_Status', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv',
       'NAICS_digit'],
      dtype='object')

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, test_size=0.05, random_state=42, stratify= y)

cat_col = ['RevLineCr', 'LowDoc', 'FranchiseCode', 'NAICS_digit', 'NewExist', 'UrbanRural' ]
num_col = ['Term', 'CreateJob', 'RetainedJob', 'SBA_Appv', 'GrAppv']

In [56]:
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [38]:
# random search cv
hyper_grid = {'classifier__max_depth':list(np.arange(10, 100, step=10)) + [30],
              'classifier__n_estimators':[100],
              'classifier__max_features':randint(1,7),
              'classifier__min_samples_leaf':randint(1,4),
              'classifier__min_samples_split':np.arange(2, 10, step=2)
          }


preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_col),
        ('num', num_transformer, num_col)
    ])

# Création du pipeline : prétraitement + modèle RandomForest
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier())])

In [ ]:
random_cv = RandomizedSearchCV(estimator= pipeline,
                               param_distributions=hyper_grid,
                               cv = 3,
                               n_iter= 9,
                               scoring = 'accuracy',
                               n_jobs= None,
                               return_train_score = True,
                               random_state = 42)

random_cv.fit(X_train, y_train)

# Afficher les meilleurs paramètres
print("Meilleurs paramètres:", random_cv.best_params_)

# Performance du meilleur modèle trouvé
score_tr = random_cv.best_estimator_.score(X_train, y_train)
score_te = random_cv.best_estimator_.score(X_test, y_test)

print("Score du modèle (train) :", score_tr)
print("Score du modèle (test) :", score_te)


# #modele sans randomizedsearchcv
# #pipeline.fit(X_train, y_train)

# # performance du modèle
# score_tr = pipeline.score(X_train, y_train)
# score_te = pipeline.score(X_test, y_test)

# print("Score du modèle (train) :", score_tr)
# print("Score du modèle (test) :", score_te)




RandomizedSearchCV donne les résultats suivants:
Meilleurs paramètres: {'classifier__max_depth': 70, 'classifier__max_features': 4, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 6, 'classifier__n_estimators': 100}
Score du modèle (train) : 0.824524119140504
Score du modèle (test) : 0.8231104472953844

On peut restreindre la recherche d'hyperparamètres à des valeurs proches de ces résultats dans la gridsearchcv.


In [65]:
# on garde la même configuration des X et y.

# GridSearchCv
params = {'classifier__max_depth': [60, 70, 80],
              'classifier__n_estimators':[90, 100, 110],
              'classifier__max_features': [3, 4, 5],
              'classifier__min_samples_leaf':[1,2],
              'classifier__min_samples_split': [5,6,7]
          }


preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_col),
        ('num', num_transformer, num_col)
    ])

# Création du pipeline : prétraitement + modèle RandomForest
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier())])

In [66]:
grid = GridSearchCV(pipeline, param_grid = params, scoring = 'accuracy', cv = 4)

grid.fit(X_train, y_train)
print("Meilleurs paramètres de GridSearch:", grid.best_params_)


# Performance du meilleur modèle trouvé
score_tr = grid.best_estimator_.score(X_train, y_train)
score_te = grid.best_estimator_.score(X_test, y_test)

print("Score du modèle (train) :", score_tr)
print("Score du modèle (test) :", score_te)


### Hyperparamétres du modéle

* Random Forest
     - max_depth
     - min_sample_split
     - max_leaf_nodes
     - min_samples_leaf
     - n_estimators
     - max_sample (bootstrap sample)
     - max_features


In [ ]:

# random search cv
hyper_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],  # Ajoutez les stratégies d'imputation numérique ici si nécessaire
    'classifier_max_depth': list(np.arange(10, 100, step=10)) + [None],
    'classifier__n_estimators': np.arange(10, 500, step=50),
    'classifier__max_features': randint(1, 7),
    'classifier__criterion': ['gini', 'entropy'],
    'classifier__min_samples_leaf': randint(1, 4),
    'classifier__min_samples_split': np.arange(2, 10, step=2)
}

# Création du preprocessor pour gérer les deux types de colonnes
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_col),
        ('num', num_transformer, num_col)
    ])

# Création du pipeline incluant le prétraitement et le modèle RandomForest
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier())])

random_cv = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=hyper_grid,
    cv=3,
    n_iter=5,
    scoring='accuracy',  # ou une autre métrique que vous souhaitez évaluer
    n_jobs=-1,
    return_train_score=True,
    random_state=42
)

# Fit du RandomizedSearchCV
random_cv.fit(X_train, y_train)

# Afficher les meilleurs paramètres
print("Meilleurs paramètres:", random_cv.best_params_)

# Performance du meilleur modèle trouvé
score_tr = random_cv.best_estimator_.score(X_train, y_train)
score_te = random_cv.best_estimator_.score(X_test, y_test)

print("Score du modèle (train) :", score_tr)
print("Score du modèle (test) :", score_te)


#### Boosting

##### Adaboost

In [8]:
#adaboost

from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, classification_report

#separer dataset en features et target
X = df.drop('MIS_Status', axis=1)
y = df['MIS_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, test_size=0.05, random_state=42, stratify=y)

cat_col = ['RevLineCr', 'LowDoc', 'FranchiseCode' ]
num_col = ['NAICS_digit', 'Term', 'NewExist', 'UrbanRural']

cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])

In [39]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_col),
        ('num', num_transformer, num_col)
    ])

# Création du pipeline : prétraitement + modèle RandomForest
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', AdaBoostClassifier())])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

# performance du modèle
score_tr = pipeline.score(X_train, y_train)
score_te = pipeline.score(X_test, y_test)

print("Score du modèle (train) :", score_tr)
print("Score du modèle (test) :", score_te)

#matrice confusion
conf_matrix = confusion_matrix(y_test, y_pred)

print("Métrique pour le modèle AdaboostClassifierr")
print("Score d'accuracy", accuracy_score(y_test, y_pred))
print("Score du recall : ", recall_score(y_test, y_pred))
print("Score de la precision : ", precision_score(y_test, y_pred))
print("Score F1 : ", f1_score(y_test, y_pred))
print("")
print(classification_report(y_test, y_pred))

Score du modèle (train) : 0.8935711189547341
Score du modèle (test) : 0.894044606930682
Métrique pour le modèle XGBClassifier
Score d'accuracy 0.894044606930682
Score du recall :  0.5917454316320101
Score de la precision :  0.7562409405701401
Score F1 :  0.663956447963801

              precision    recall  f1-score   support

           0       0.92      0.96      0.94     73847
           1       0.76      0.59      0.66     15870

    accuracy                           0.89     89717
   macro avg       0.84      0.78      0.80     89717
weighted avg       0.89      0.89      0.89     89717



#### XGBoost

In [30]:
from xgboost import XGBClassifier

cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop = 'if_binary'))
])

num_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors = 3, weights = 'distance')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_col),
        ('num', num_transformer, num_col)
    ])

# Création du pipeline : prétraitement + modèle XGBoost
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', XGBClassifier(random_state=42))])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

# performance du modèle
score_tr = pipeline.score(X_train, y_train)
score_te = pipeline.score(X_test, y_test)

print("Score du modèle (train) :", score_tr)
print("Score du modèle (test) :", score_te)

#matrice confusion
conf_matrix = confusion_matrix(y_test, y_pred)

print("Métrique pour le modèle XGBClassifier")
print("Score d'accuracy", accuracy_score(y_test, y_pred))
print("Score du recall : ", recall_score(y_test, y_pred))
print("Score de la precision : ", precision_score(y_test, y_pred))
print("Score F1 : ", f1_score(y_test, y_pred))
print("")
print(classification_report(y_test, y_pred))


/home/utilisateur/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Score du modèle (train) : 0.930283418670246
Score du modèle (test) : 0.9302035266055864
Métrique pour le modèle XGBClassifier
Score d'accuracy 0.9302035266055864
Score du recall :  0.765041888804265
Score de la precision :  0.8248255097851376
Score F1 :  0.793809680605861

              precision    recall  f1-score   support

           0       0.95      0.97      0.96     36981
           1       0.82      0.77      0.79      7878

    accuracy                           0.93     44859
   macro avg       0.89      0.87      0.88     44859
weighted avg       0.93      0.93      0.93     44859



/home/utilisateur/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Rappel de l'encodage pour la variable MIS_Status: {'P I F': 0, 'CHGOFF': 1}



#### CatBoost

In [34]:
#separer dataset en features et target
X = df_cat.drop('MIS_Status', axis=1)
y = df_cat['MIS_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, test_size=0.05, random_state=42, stratify= y)

cat_col = ['RevLineCr', 'LowDoc', 'FranchiseCode', 'UrbanRural']
num_col = ['NAICS_digit', 'Term', 'SBA_Appv', 'GrAppv']

In [33]:
df_cat = df.drop(['LoanNr_ChkDgt', 'Name','NAICS', 'City', 'State', 'Zip', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY', 'NoEmp', 'CreateJob', 'RetainedJob', 'ChgOffDate', 'DisbursementDate', 'DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'NewExist'], axis=1)
df_cat.dtypes

Term               int64
FranchiseCode      int64
UrbanRural         int64
RevLineCr         object
LowDoc            object
MIS_Status         int64
GrAppv           float64
SBA_Appv         float64
NAICS_digit       object
dtype: object

In [35]:
from catboost import CatBoostClassifier
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop = 'if_binary'))
])

num_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors = 2, weights = 'uniform')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_col),
        ('num', num_transformer, num_col)
    ])

# Création du pipeline : prétraitement + modèle CatBoost
modelcat = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', CatBoostClassifier(random_state=42, verbose=False))])

modelcat.fit(X_train, y_train)

y_pred = modelcat.predict(X_test)

# performance du modèle
score_tr = modelcat.score(X_train, y_train)
score_te = modelcat.score(X_test, y_test)

print("Score du modèle (train) :", score_tr)
print("Score du modèle (test) :", score_te)

#matrice confusion
conf_matrix = confusion_matrix(y_test, y_pred)

#extrait classifieur de la pipeline
catboost_classifier = modelcat.named_steps['classifier']
#feature importance
feature_importance = catboost_classifier.get_importance(prettified=True)
print('importance des features: feature_importance)
#feature_importance = pipeline.named_steps['classifier'].get_feature_importance()

print("Métrique pour le modèle XGBClassifier")
print("Score d'accuracy", accuracy_score(y_test, y_pred))
print("Score du recall : ", recall_score(y_test, y_pred))
print("Score de la precision : ", precision_score(y_test, y_pred))
print("Score F1 : ", f1_score(y_test, y_pred))
print("")
print(classification_report(y_test, y_pred))


Score du modèle (train) : 0.9407315195915091
Score du modèle (test) : 0.9381840879199269
Noms des variables et leur importance : {'RevLineCr_Y': 'Feature Id', 'LowDoc_Y': 'Importances'}
Métrique pour le modèle XGBClassifier
Score d'accuracy 0.9381840879199269
Score du recall :  0.8010916476263011
Score de la precision :  0.8395636557137156
Score F1 :  0.8198765833062682

              precision    recall  f1-score   support

           0       0.96      0.97      0.96     36981
           1       0.84      0.80      0.82      7878

    accuracy                           0.94     44859
   macro avg       0.90      0.88      0.89     44859
weighted avg       0.94      0.94      0.94     44859



In [31]:
# Obtenez les noms de colonnes transformées
preprocessed_cat_columns = modelcat.named_steps['preprocessor'].transformers_[0][1]['onehot'].get_feature_names_out(input_features=cat_col)
preprocessed_columns = list(preprocessed_cat_columns) + num_col

print('Noms des variables :', preprocessed_columns)



Noms des variables : ['RevLineCr_Y', 'LowDoc_Y', 'FranchiseCode_1', 'UrbanRural_0', 'UrbanRural_1', 'UrbanRural_2', 'NewExist_0', 'NewExist_1', 'NewExist_2', 'NAICS_digit', 'Term', 'SBA_Appv', 'GrAppv']


In [ ]:
from catboost import CatBoostClassifier
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop = 'if_binary'))
])

num_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors = 2, weights = 'uniform')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_col),
        ('num', num_transformer, num_col)
    ])

# Création du pipeline : prétraitement + modèle CatBoost
modelcat = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', CatBoostClassifier(random_state=42, verbose=False))])

modelcat.fit(X_train, y_train)

y_pred = modelcat.predict(X_test)

# performance du modèle
score_tr = modelcat.score(X_train, y_train)
score_te = modelcat.score(X_test, y_test)

print("Score du modèle (train) :", score_tr)
print("Score du modèle (test) :", score_te)

#matrice confusion
conf_matrix = confusion_matrix(y_test, y_pred)

# #extrait classifieur de la pipeline
# catboost_classifier = modelcat.named_steps['classifier']
# #feature importance
# feature_importance = catboost_classifier.get_feature_importance(prettified=True)
# print('importance des features:', feature_importance)
# #feature_importance = pipeline.named_steps['classifier'].get_feature_importance()

print("Métrique pour le modèle XGBClassifier")
print("Score d'accuracy", accuracy_score(y_test, y_pred))
print("Score du recall : ", recall_score(y_test, y_pred))
print("Score de la precision : ", precision_score(y_test, y_pred))
print("Score F1 : ", f1_score(y_test, y_pred))
print("")
print(classification_report(y_test, y_pred))


In [52]:
#le modele catboost est le plus performant pour l'instant, on le charge sous forme d'un fichier pkl
import pickle

pickle.dump(modelcat, open('modelcatb.pkl', 'wb'))

In [40]:
df_cat.columns
df_cat.head(5)

,Term,FranchiseCode,UrbanRural,RevLineCr,LowDoc,MIS_Status,GrAppv,SBA_Appv,NAICS_digit
0,84,0,0,N,Y,0,60000.0,48000.0,45
1,60,0,0,N,Y,0,40000.0,32000.0,72
2,180,0,0,N,N,0,287000.0,215250.0,62
3,60,0,0,N,Y,0,35000.0,28000.0,0
4,240,0,0,N,N,0,229000.0,229000.0,0


In [46]:
# values = {'Term':(120,100), 'FranchiseCode': (0,1), 'UrbanRural': (0,0), 'RevLineCr': ('N','Y'), 'LowDoc': ('Y','N'), 'GrAppv': (48000, 28000), 'SBA_Appv':(20000,60000), 'NAICS_digit':(45,62)}
# pred = modelcat.predict(values)

print(X)

        Term  FranchiseCode  UrbanRural RevLineCr LowDoc    GrAppv  SBA_Appv  \
0         84              0           0         N      Y   60000.0   48000.0   
1         60              0           0         N      Y   40000.0   32000.0   
2        180              0           0         N      N  287000.0  215250.0   
3         60              0           0         N      Y   35000.0   28000.0   
4        240              0           0         N      N  229000.0  229000.0   
...      ...            ...         ...       ...    ...       ...       ...   
897162    60              0           0       NaN      N   70000.0   56000.0   
897163    60              0           0         Y      N   85000.0   42500.0   
897164   108              0           0         N      N  300000.0  225000.0   
897165    60              0           0         N      Y   75000.0   60000.0   
897166    48              0           0         N      N   30000.0   24000.0   

       NAICS_digit  
0               45

In [55]:
def predict_loan(model,data):
    # Convertir la liste de listes en DataFrame
    df = pd.DataFrame(data, columns=['Term', 'FranchiseCode', 'UrbanRural', 'RevLineCr', 'LowDoc', 'GrAppv', 'SBA_Appv', 'NAICS_digit'])
    predictions = model.predict(df)
    return predictions[0]

predict_loan(modelcat,[[84, 0, 0, 'N', 'Y',60000.0, 48000.0, 45]])

0

In [58]:
print(y[897166])
y

0


0         0
1         0
2         0
3         0
4         0
         ..
897162    0
897163    0
897164    0
897165    1
897166    0
Name: MIS_Status, Length: 897167, dtype: int64

#### Stacking/Voting